In [1]:
# Setup

import numpy as np
import math
#import matplotlib.pyplot as plt
import sys
#import math
import pandas as pd

from bokeh.plotting import figure, show, output_notebook
from bokeh.models import GlyphRenderer, LinearColorMapper, LinearAxis, Range1d
from bokeh.io import push_notebook
#from numba import jit, njit

from ipywidgets import interact

from scipy import optimize
from scipy.signal import savgol_filter

output_notebook()

tools = "hover, box_zoom, undo, crosshair, save, pan, reset, wheel_zoom"

Loading BokehJS ...

In [2]:
# Data Acq

logData2 = pd.read_csv("24jan.csv") #("Log.csv")#("Log_File_27_Nov.csv")
logData2.head(5)

,Time,Vaisala Hum,Vaisala Temp,SGS Hum,SGS Temp,SGS Raw
0,11:58:20,88.3,16.3,86.7,16.8,469636
1,11:58:21,88.4,16.3,86.7,16.8,469619
2,11:58:22,88.4,16.3,86.7,16.8,469617
3,11:58:23,88.4,16.3,86.7,16.8,469616
4,11:58:24,88.4,16.3,86.7,16.8,469609


In [3]:
def multi_ys(x, y1, y11, y2):
    p = figure(title="Raw Data and Humidity Variation", x_axis_label='Time(sec)', y_axis_label='Raw', width=950, tools=tools)

    p.y_range = Range1d(y1.min(), y1.max())
    p.extra_y_ranges = {"y2_range": Range1d(y2.min(), y2.max())}
    p.add_layout(LinearAxis(y_range_name="y2_range"), "right")

    p.line(x, y1, name="Data", line_width=2, color="red", legend_label="Y1")
    p.line(x, y11, name="Data", line_width=2, color="orange", legend_label="Y1")

    p.line(x, y2, name="Data", line_width=2, color="blue", legend_label="Y2", y_range_name="y2_range")

    show(p)

In [4]:
def multi1_ys(x, y1, y2):
    p = figure(title="Raw Data and Humidity Variation", x_axis_label='Time(sec)', y_axis_label='Raw', width=950, tools=tools)

    p.y_range = Range1d(y1.min() - 5, y1.max() + 5)
    p.extra_y_ranges = {"y2_range": Range1d(y2.min(), y2.max())}
    p.add_layout(LinearAxis(y_range_name="y2_range"), "right")

    p.line(x, y1, name="Data", line_width=2, color="red", legend_label="Y1")

    p.line(x, y2, name="Data", line_width=2, color="blue", legend_label="Y2", y_range_name="y2_range")

    show(p)

In [5]:
# Simple Data Comaprison

log_data2 = logData2.to_numpy()

# Positions in Table
_vhum = 1
_vtemp = 2
_shum = 3
_stemp = 4
_sraw = 5

#

vhum = log_data2[1:,_vhum]    # Vaisala Hum,
vtemp = log_data2[1:,_vtemp]  # Vaisala Temp,
stemp = log_data2[1:,_stemp]  # SGS Temp,
shum = log_data2[1:,_shum]    # SGS Hum,
sraw = log_data2[1:,_sraw]    # SGS Hum,


count = [ i for i in range(len(vhum)) ]

In [6]:
p = figure(title="Raw Data ", x_axis_label='Time (Sec)', y_axis_label='SGS Hum', width=950, tools=tools)
p.line(count, vtemp, legend_label="V Hum", color="blue", line_width=2)
p.line(count, stemp, legend_label="S Hum", color="orange", line_width=2)
show(p)

In [7]:
diff_arr = np.array([0])

_count = -1
for x in vtemp:
    _count = _count + 1
    diff = vtemp[_count] - stemp[_count] 
    #print(diff)
    diff_arr = np.append(diff_arr, [diff], axis=0)

print(diff_arr)

p = figure(title="Temperature Diff ", x_axis_label='Time (Sec)', y_axis_label='SGS Temp', width=950, tools=tools)
p.line(count, diff_arr, legend_label="Diff", color="blue", line_width=2)
show(p)

[ 0.  -0.5 -0.5 ... -0.5 -0.5 -0.5]


In [8]:
# Humidity Diff

diff_arr = np.array([0])

_count = -1
for x in vhum:
    _count = _count + 1
    diff = vhum[_count] - shum[_count]
    #print(diff)
    diff_arr = np.append(diff_arr, [diff], axis=0)

print(diff_arr)

p = figure(title="Temperature Diff ", x_axis_label='Time (Sec)', y_axis_label='SGS Temp', width=950, tools=tools)
p.line(count, diff_arr, legend_label="Diff", color="blue", line_width=2)
show(p)

[0.  1.7 1.7 ... 3.4 3.4 3.4]


In [9]:
p = figure(title="Raw Data ", x_axis_label='Time (Sec)', y_axis_label='SGS Hum', width=950, tools=tools)
p.line(count, vhum, legend_label="V Hum", color="blue", line_width=2)
p.line(count, shum, legend_label="S Hum", color="orange", line_width=2)
show(p)

In [10]:
 multi_ys(count, shum, vhum, diff_arr)

In [11]:
p = figure(title="Raw Data ", x_axis_label='Hum', y_axis_label='Difference', width=950, tools=tools)
p.line(vhum, diff_arr, color="blue", line_width=2)
show(p)

In [12]:
 multi_ys(count, shum, vhum, diff_arr)

In [13]:
multi1_ys(count,diff_arr,vhum)

In [14]:
multi1_ys(count,vhum, stemp)

In [15]:
multi1_ys(count,vhum, sraw)

In [16]:
# Humidity Diff

diff_arr = np.array([0])

_count = -1
for x in vhum:
    _count = _count + 1
    diff = vhum[_count] - shum[_count]
    #print(diff)
    diff_arr = np.append(diff_arr, [diff], axis=0)

print(diff_arr)

p = figure(title="Humidity Diff ", x_axis_label='Time (Sec)', y_axis_label='SGS Temp', width=950, tools=tools)
p.line(count, diff_arr, legend_label="Diff", color="blue", line_width=2)
show(p)

[0.  1.7 1.7 ... 3.4 3.4 3.4]


In [17]:
diff_arr_avgd = np.array([0])

_count = 10
for x in vhum:
    if _count < 31730:
        #print(_count)
        _count = _count + 1
        '''diff = (
            diff_arr[_count - 6] + 
            diff_arr[_count - 5] + 
            diff_arr[_count - 4] + 
            diff_arr[_count - 3] + 
            diff_arr[_count - 2] + 
            diff_arr[_count - 1] + 
            diff_arr[_count] + 
            diff_arr[_count + 1] + 
            diff_arr[_count + 2] + 
            diff_arr[_count + 3] + 
            diff_arr[_count + 4] + 
            diff_arr[_count + 5]
            ) / 12'''
        diff_arr_avgd = np.append(diff_arr_avgd, [diff], axis=0)

print(diff_arr_avgd)

yhat = savgol_filter(diff_arr, 2501, 3)

p = figure(title="Temperature Diff ", x_axis_label='Time (Sec)', y_axis_label='SGS Temp', width=950, tools=tools)
p.line(count, yhat, legend_label="Diff", color="blue", line_width=2)
p.line(count, diff_arr, legend_label="Diff", color="orange", line_width=2)
show(p)

[0.  3.4 3.4 ... 3.4 3.4 3.4]
